In [1]:
from requests import post
from app.analysis import Quote
query = 'query UnknownFile($limit_0:[FundLimitOptionInput!]!) {viewer {...Fe}} fragment Fe on Viewer {_screenFunds4s9d1a:screenFunds(filterCategory:["$$"],limit:$limit_0,first:100) {edges {node {symbol}}}}'
variables = {'limit_0':[{'type':'FUND_TOTAL_ASSETS','min':2000000000,'inclusive':True},{'type':'FUND_NET_EXPENSE_RATIO','max':.5,'inclusive':True}]}
r = post('https://lens.m1finance.com/graphql', json={'query': query.replace('$$', 'Equities","Domestic'), 'variables': variables}).json()
symbols = [n['node']['symbol'] for n in r['data']['viewer']['_screenFunds4s9d1a']['edges']]

In [2]:
quote = Quote(symbols, 120)
f'from {quote.start.date()} to {quote.end.date()} - {len(quote.data.columns)}'

'from 2019-03-04 to 2019-06-28 - 86'

In [3]:
stat = quote.statistics(21)
stat = stat[(stat['drawdown'] < 8) & (stat['21-shrp'] > 0.4)]
stat

,21-len,21-mean,21-std,21-shrp,drawdown
Symbols,,,,,
PGX,62,0.792361,0.436035,1.434961,1.237116
SPLV,62,2.100320,1.418272,1.363386,2.660443
USMV,62,2.159025,1.769566,1.125902,2.180427
MTUM,62,2.128084,2.127335,0.922006,2.859384
PFF,62,0.782232,0.674081,0.913192,1.003023
XMLV,62,1.338605,1.580594,0.741454,3.688597
VIG,62,1.773538,2.542372,0.632036,4.634193
VOT,62,1.641305,3.226571,0.457029,5.627906
VOOG,62,1.434180,2.935914,0.431727,6.391441


In [4]:
quote.setup_mask(stat.index)
quote.least_correlated_portfolio(21, 2)

['PGX'] -0.38753817198258056 0.6 0.24742325575833032 1.434961427740911
['PGX', 'SPLV'] -0.5240759022552762 0.4853419746047747 0.38975592885216226 1.3991738057122132


['PGX', 'SPLV']

In [5]:
quote.setup_mask(['PGX', 'SPLV'])
quote.find_optimal_ratio(21, 2)

/Users/chaoyi/.local/share/virtualenvs/portfolio_analysis-bdMx2S3t/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


({'PGX': 1.07, 'SPLV': -0.07}, 0.7028, 0.4273)